In [74]:
# import libraries
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import os
import numpy as np
import re
import random
import nltk
from scipy import sparse
from scipy.sparse import csr_matrix, vstack
from textblob import TextBlob
from langdetect import detect_langs
import pickle
from datetime import datetime

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [75]:
song_df = pd.read_csv('./fake_job_postings.csv')
song_df

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17875,17876,Account Director - Distribution,"CA, ON, Toronto",Sales,NaN,Vend is looking for some awesome new talent to...,Just in case this is the first time you’ve vis...,To ace this role you:Will eat comprehensive St...,What can you expect from us?We have an open cu...,0,1,1,Full-time,Mid-Senior level,NaN,Computer Software,Sales,0
17876,17877,Payroll Accountant,"US, PA, Philadelphia",Accounting,NaN,WebLinc is the e-commerce platform and service...,The Payroll Accountant will focus primarily on...,- B.A. or B.S. in Accounting- Desire to have f...,Health &amp; WellnessMedical planPrescription ...,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Internet,Accounting/Auditing,0
17877,17878,Project Cost Control Staff Engineer - Cost Con...,"US, TX, Houston",NaN,NaN,We Provide Full Time Permanent Positions for m...,Experienced Project Cost Control Staff Enginee...,At least 12 years professional experience.Abil...,NaN,0,0,0,Full-time,NaN,NaN,NaN,NaN,0
17878,17879,Graphic Designer,"NG, LA, Lagos",NaN,NaN,NaN,Nemsia Studios is looking for an experienced v...,1. Must be fluent in the latest versions of Co...,Competitive salary (compensation will be based...,0,0,1,Contract,Not Applicable,Professional,Graphic Design,Design,0


In [76]:
song_df.fillna(" ",inplace = True)
song_df['text'] =  song_df['title'] + ' ' + song_df['location'] + ' ' + song_df['department'] + ' ' + song_df['company_profile'] + ' ' + song_df['description'] + ' ' + song_df['requirements'] + ' ' + song_df['benefits'] + ' ' + song_df['employment_type'] + ' ' + song_df['required_education'] + ' ' + song_df['industry'] + ' ' + song_df['function'] 
song_df.rename(columns = {"fraudulent": "artist"}, inplace=True)
song_df.reset_index(drop=True, inplace=True)


del song_df['job_id']
del song_df['title']
del song_df['salary_range']
del song_df['location']
del song_df['department']
del song_df['company_profile']
del song_df['description']
del song_df['requirements']
del song_df['benefits']
del song_df['employment_type']
del song_df['required_experience']
del song_df['required_education']
del song_df['industry']
del song_df['function']
del song_df['telecommuting']
del song_df['has_questions']
del song_df['has_company_logo']
song_df

,artist,text
0,0,"Marketing Intern US, NY, New York Marketing We..."
1,0,"Customer Service - Cloud Video Production NZ, ..."
2,0,"Commissioning Machinery Assistant (CMA) US, IA..."
3,0,"Account Executive - Washington DC US, DC, Wash..."
4,0,"Bill Review Manager US, FL, Fort Worth SpotS..."
...,...,...
17875,0,"Account Director - Distribution CA, ON, Toron..."
17876,0,"Payroll Accountant US, PA, Philadelphia Accoun..."
17877,0,Project Cost Control Staff Engineer - Cost Con...
17878,0,"Graphic Designer NG, LA, Lagos Nemsia Stud..."


In [77]:
df = song_df[['text', 'artist']]


tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
df['tokens'] = df['text'].map(tokenizer.tokenize)

delete_line = []
for i in range(len(df)):
         if(len(df['tokens'].iloc[i])>=800):
            delete_line.append(i)
            continue
df.drop(delete_line, inplace=True)
df.reset_index(drop=True, inplace=True)
del df['tokens']

C:\Users\EIRAOI~1\AppData\Local\Temp/ipykernel_15448/2269883234.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tokens'] = df['text'].map(tokenizer.tokenize)
C:\Users\EIRAOI~1\AppData\Local\Temp/ipykernel_15448/2269883234.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(delete_line, inplace=True)


In [78]:
for i in range(len(df)):
        if(df['artist'].iloc[i]==0):
            df['artist'].iloc[i]='true_job'
        else:
            df['artist'].iloc[i]='fake_job'

df

C:\Users\EIRAOI~1\AppData\Local\Temp/ipykernel_15448/728911341.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['artist'].iloc[i]='true_job'
C:\Users\EIRAOI~1\AppData\Local\Temp/ipykernel_15448/728911341.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['artist'].iloc[i]='fake_job'


,text,artist
0,"Marketing Intern US, NY, New York Marketing We...",true_job
1,"Commissioning Machinery Assistant (CMA) US, IA...",true_job
2,"Account Executive - Washington DC US, DC, Wash...",true_job
3,"Bill Review Manager US, FL, Fort Worth SpotS...",true_job
4,"Accounting Clerk US, MD, Job OverviewApex...",true_job
...,...,...
16954,"Recruiting Coordinator US, NC, Charlotte R...",true_job
16955,"JavaScript Developer US, , Sr, JavaScript...",true_job
16956,"Payroll Accountant US, PA, Philadelphia Accoun...",true_job
16957,Project Cost Control Staff Engineer - Cost Con...,true_job


In [79]:
df.to_csv('./fakejob.csv', header=True, index=False)